Simple dataset from Kaggle: https://www.kaggle.com/neuromusic/avocado-prices/home

In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn import linear_model
from sklearn import neighbors
from scipy import stats

In [2]:
df = pd.read_csv('data/avocado.csv')

In [3]:
df.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
df.describe()

,Unnamed: 0,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year
count,18249.000000,18249.000000,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,18249.000000,18249.000000
mean,24.232232,1.405978,8.506440e+05,2.930084e+05,2.951546e+05,2.283974e+04,2.396392e+05,1.821947e+05,5.433809e+04,3106.426507,2016.147899
std,15.481045,0.402677,3.453545e+06,1.264989e+06,1.204120e+06,1.074641e+05,9.862424e+05,7.461785e+05,2.439660e+05,17692.894652,0.939938
min,0.000000,0.440000,8.456000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2015.000000
25%,10.000000,1.100000,1.083858e+04,8.540700e+02,3.008780e+03,0.000000e+00,5.088640e+03,2.849420e+03,1.274700e+02,0.000000,2015.000000
50%,24.000000,1.370000,1.073768e+05,8.645300e+03,2.906102e+04,1.849900e+02,3.974383e+04,2.636282e+04,2.647710e+03,0.000000,2016.000000
75%,38.000000,1.660000,4.329623e+05,1.110202e+05,1.502069e+05,6.243420e+03,1.107834e+05,8.333767e+04,2.202925e+04,132.500000,2017.000000
max,52.000000,3.250000,6.250565e+07,2.274362e+07,2.047057e+07,2.546439e+06,1.937313e+07,1.338459e+07,5.719097e+06,551693.650000,2018.000000


In [24]:
# make an OLS model
ols = linear_model.LinearRegression()

Y = df['AveragePrice']
#X = df[['Total Volume', 'year', '4046', '4225', '4770']]
# Lines below for normalization by std, use line above for raw values
X = pd.DataFrame({
    'Total Volume': stats.zscore(df['Total Volume']),
    'year': stats.zscore(df['year']),
    '4046': stats.zscore(df['4046']),
    '4225': stats.zscore(df['4225']),
    '4770': stats.zscore(df['4770'])
})
ols.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [25]:
from sklearn.model_selection import cross_val_score
ols_score = cross_val_score(ols, X, Y, cv=10)
print("Linear Regression average accuracy amongst 5 trials: ", ols_score.mean())

Linear Regression average accuracy amongst 5 trials:  -1.482320956162525


In [26]:
ols_score

array([-4.77528675, -2.41337402, -1.57985947, -0.24652243, -1.57091875,
       -2.19885316, -0.65071967, -0.29448442, -0.47648743, -0.61670346])

In [27]:
# make a KNN model
knn = neighbors.KNeighborsRegressor(n_neighbors=5)

knn.fit(X, Y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [28]:
knn_score = cross_val_score(knn, X, Y, cv=10)
print("KNN accuracy average amongst 5 trials: ", knn_score.mean())

KNN accuracy average amongst 5 trials:  -0.4400074149793081


In [29]:
# make a KNN model with weights
knn_w = neighbors.KNeighborsRegressor(n_neighbors=5, weights='distance')

knn_w.fit(X, Y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='distance')

In [38]:
knn_w_score = cross_val_score(knn_w, X, Y, cv=10)
print("KNN average accuracy amongst 5 trials: ", knn_w_score.mean())

KNN average accuracy amongst 5 trials:  -0.44407498014024505


In [39]:
knn_w_score

array([-0.45311054, -0.41345904, -0.4916306 , -0.02224545, -1.17355137,
       -0.38503706, -0.17950905, -0.34506331, -0.28420566, -0.69293773])